# Word Embeddings: numerical representations to classify clinical text

_Jocelyn Dunstan & Fabián Villena, Centro de Informática Médica y Telemedicina, Universidad de Chile_

Modules loading

In [ ]:
import numpy as np #linear algebra
import nltk #natural language processing tools
import gensim #neural word embedding training
import re #regular expressions
import logging #verbosity (code telling you what's going on)
import csv #data loading
import sklearn.ensemble #classifier
import sklearn.metrics #performance assessment
import sklearn.model_selection #train-test split

Downloading extra data for nltk

In [ ]:
nltk.download('stopwords') # connectors and other non-semantic rich expressions
nltk.download('punkt') # sentence tokenizer

Adding some verbosity to the embedding training session, i.e., the coding telling you what is doing

In [6]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Loading spanish stopwords list from nltk

In [7]:
stopwords = nltk.corpus.stopwords.words('spanish')

In [8]:
set(stopwords)

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

Some self-defined functions

In [9]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

In [10]:
def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.mean(vectors,axis=0))

## Word Embedding

Loading the corpus dataset and taking each line to an array of sentences. We are using the whole Aysen's waiting list as a corpus

In [11]:
sentences = []
with open('corpus.txt', encoding='utf-8') as file: #use utf-8 to preserve special characters
    for line in file:
        sentences.append((line.rstrip())) #use strip to remove \n (newline)

Taking every sentence and normalizing it

In [12]:
normalized_sentences = [normalizer(sentence) for sentence in sentences]

Tokenizing every sentence into words

In [13]:
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in normalized_sentences]

Removing stopwords

In [14]:
without_stopwords_sentences = []
for sentence in tokenized_sentences:
    without_stopwords_sentence = [word for word in sentence if word not in stopwords]
    without_stopwords_sentences.append(without_stopwords_sentence)

Training the neural word embeddings word2vec

In [15]:
model = gensim.models.Word2Vec(without_stopwords_sentences)

2019-06-15 10:27:44,803 : INFO : collecting all words and their counts
2019-06-15 10:27:44,806 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-15 10:27:44,839 : INFO : PROGRESS: at sentence #10000, processed 26393 words, keeping 2174 word types
2019-06-15 10:27:44,861 : INFO : PROGRESS: at sentence #20000, processed 53049 words, keeping 2795 word types
2019-06-15 10:27:44,883 : INFO : PROGRESS: at sentence #30000, processed 80100 words, keeping 3189 word types
2019-06-15 10:27:44,908 : INFO : PROGRESS: at sentence #40000, processed 107473 words, keeping 3528 word types
2019-06-15 10:27:44,931 : INFO : PROGRESS: at sentence #50000, processed 134970 words, keeping 3783 word types
2019-06-15 10:27:44,956 : INFO : PROGRESS: at sentence #60000, processed 162957 words, keeping 3986 word types
2019-06-15 10:27:44,985 : INFO : PROGRESS: at sentence #70000, processed 191225 words, keeping 4140 word types
2019-06-15 10:27:45,024 : INFO : PROGRESS: at sentence #

Retrieving the most similar words to a given word

In [16]:
model.wv.most_similar('diente')

2019-06-15 10:27:51,692 : INFO : precomputing L2-norms of word weight vectors


[('arcos', 0.9807857275009155),
 ('posicion', 0.9750917553901672),
 ('dentarios', 0.9700753688812256),
 ('relacion', 0.9640514850616455),
 ('evidentes', 0.9584258794784546),
 ('anomalias', 0.9575490951538086),
 ('dentofaciales', 0.9548975229263306),
 ('maxilobasilar', 0.9412839412689209),
 ('tamaño', 0.9192438721656799),
 ('incluso', 0.9082833528518677)]

Test your own words!

## Classifier: Can we classify the medical specialty from the diagnosis?

Loading the raw dataset and extracting the features and labels

In [17]:
diagnostics = [] #classifier raw features
specialties = [] #classifier raw labels
with open('data.csv', encoding='utf-8') as file:
    data = csv.DictReader(file)
    for row in data:
        diagnostics.append(row['diagnostic'])
        specialties.append(row['specialty'])

Preprocessing

In [18]:
diagnostics_normalized = [normalizer(diagnostic) for diagnostic in diagnostics]

In [19]:
diagnostics_tokenized = [nltk.word_tokenize(diagnostic) for diagnostic in diagnostics_normalized]

In [20]:
diagnostics_wihout_stopwords = []
for diagnostic in diagnostics_tokenized:
    diagnostic_wihout_stopwords = [word for word in diagnostic if word not in stopwords]
    diagnostics_wihout_stopwords.append(diagnostic_wihout_stopwords)

Creating an empty matrix to store the encoded features

In [21]:
diagnostics_matrix = np.zeros((len(diagnostics_wihout_stopwords), len(model.wv['cancer'])))

Filling the matrix with the vectorized diagnostics

In [22]:
for i,diagnostic in enumerate(diagnostics_wihout_stopwords):
    vector = vectorizer(diagnostic,model)
    diagnostics_matrix[i,] = vector

C:\Users\ville\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\ville\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Creating an empty vector to store the encoded labels

In [23]:
specialties_vector = np.zeros((len(specialties), 1))

Filling the vector with the encoded specialties

In [24]:
for i,specialty in enumerate(specialties): #OFTALMOLOGIA is encoded as a 1.0 and TRAUMATOLOGIA as a 2.0
    if specialty == 'OFTALMOLOGIA':
        specialties_vector[i,] = 1
    else:
        specialties_vector[i,] = 2

Concatenating the encoded features ans labels

In [25]:
data_matrix = np.concatenate([diagnostics_matrix,specialties_vector], axis=1)

Removing NAs from the matrix

In [26]:
data_matrix_without_nan = data_matrix[~np.isnan(data_matrix).any(axis=1)]

Splitting the dataset into training and testing subsets

In [27]:
diagnostics_train, diagnostics_test, specialties_train, specialties_test = sklearn.model_selection.train_test_split(
    data_matrix_without_nan[:,:100],
    data_matrix_without_nan[:,100],
    test_size=0.33,
    random_state=42
)

Training a Random Forest Classifier

In [28]:
classifier = sklearn.ensemble.RandomForestClassifier()

In [29]:
classifier.fit(diagnostics_train,specialties_train)

C:\Users\ville\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Making predictions over the testing subset and measuring their performance

In [30]:
predictions = classifier.predict(diagnostics_test)
print(sklearn.metrics.classification_report(predictions, specialties_test))

              precision    recall  f1-score   support

         1.0       0.78      0.98      0.87      5619
         2.0       0.98      0.82      0.89      8536

   micro avg       0.88      0.88      0.88     14155
   macro avg       0.88      0.90      0.88     14155
weighted avg       0.90      0.88      0.88     14155



- Precision = TP/(TP+FP) # accuracy of positive predictions 
- Recall = TP/(TP+FN) # sensitivity or true positive rate
- F1-score = 2 x (precision x recall)/(precision + recall)

## Manual testing of the classifier

In [31]:
def specialtyClassifier(diagnostic):
    try:
        stringNorm = normalizer(diagnostic)
        stringTokenized = nltk.word_tokenize(stringNorm)
        stringVec = vectorizer(stringTokenized,model)
        result = classifier.predict(stringVec.reshape(1, -1))[0]
        prob = classifier.predict_proba(stringVec.reshape(1, -1))[0]
        if result==1.0: 
            result = "Oftalmología"
        if result==2.0: 
            result = 'Traumatología'
        prob_oft = prob[0]
        prob_tra = prob[1]
        return('''
        
        Diagnostic: {diagnostic}
        ---
        Predicted Specialty:\t{result}
        ---
        Probabilities
        Oftalmología:\t{prob_oft}
        Traumatología:\t{prob_tra}
        
        '''.format(
            diagnostic = diagnostic,
            result = result,
            prob_oft = prob_oft,
            prob_tra = prob_tra,
        )
              )
    except:
        return('Error')

In [32]:
print(specialtyClassifier('vicio de refracción'))


        
        Diagnostic: vicio de refracción
        ---
        Predicted Specialty:	Oftalmología
        ---
        Probabilities
        Oftalmología:	1.0
        Traumatología:	0.0
        
        


In [39]:
print(specialtyClassifier('fractura de órbita'))


        
        Diagnostic: fractura de órbita
        ---
        Predicted Specialty:	Traumatología
        ---
        Probabilities
        Oftalmología:	0.4
        Traumatología:	0.6
        
        


Try your own!